# Use Python to Find Optimum Vacation Location

## Analyze Custom Cities df Based on Weather Preferences.

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/city_data_df.csv")

# Display sample data
city_data_df.head()



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,12.40,84,100,2.05,ZA,1717732087
1,1,remire-montjoly,4.9167,-52.2667,26.02,100,0,0.00,GF,1717731990
2,2,isafjordur,66.0755,-23.1240,1.20,94,98,8.53,IS,1717732087
3,3,ancud,-41.8697,-73.8203,10.71,91,82,4.42,CL,1717732087
4,4,puerto natales,-51.7236,-72.4875,-0.73,96,98,1.08,CL,1717732087


## Use hvplot to map the cities in the data frame scaled by a weather condition (humidity). 

In [3]:
#Create a geo map of the cities in the city data frame scaled by humidity

humidty_wrld = city_data_df[['Humidity', 'Lat', 'Lng', 'City']]


humidty_wrld.hvplot.points("Lng", "Lat", \
                           geo = True,\
                           tiles = "OSM",\
                           frame_width = 800, \
                           frame_height = 600, \
                           size = "Humidity", \
                           color = 'City',\
                           scale = 1.0, title="M")



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

## Create a data frame with my personal desired vacation weather, and find the places that fit that criteria. 

In [4]:
#Create a data frame with my most desirable vacation weather conditions

perfect_vacay = city_data_df.loc[(city_data_df['Max Temp'] < 30) & 
                                (city_data_df['Max Temp'] > 15) &
                                (city_data_df['Wind Speed'] < 9.5) &
                                (city_data_df['Cloudiness'] < 40) &
                                (city_data_df['Humidity'] < 73)]
perfect_vacay

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,oskaloosa,41.2964,-92.6444,20.18,48,0,4.12,US,1717732088
16,16,alice springs,-23.7000,133.8833,15.98,36,0,5.85,AU,1717732089
18,18,kununurra,-15.7667,128.7333,23.63,24,0,7.59,AU,1717732089
21,21,ust'-kut,56.7938,105.7672,17.31,27,32,2.56,RU,1717732090
29,29,ntcheu,-14.8203,34.6359,15.35,64,1,0.39,MW,1717732091
...,...,...,...,...,...,...,...,...,...,...
513,513,sidrolandia,-20.9319,-54.9614,19.18,49,0,2.30,BR,1717732163
534,534,bengkulu,-3.8004,102.2655,29.23,70,10,2.16,ID,1717732166
559,559,pokhara,28.2333,83.9833,27.35,50,27,2.15,NP,1717732170
565,565,rikaze,29.2500,88.8833,16.63,27,17,1.47,CN,1717732171


In [5]:
#Create a copy of the perfect weather df for cleaning
perfect_hotel_df = perfect_vacay.copy()
perfect_hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,oskaloosa,41.2964,-92.6444,20.18,48,0,4.12,US,1717732088
16,16,alice springs,-23.7000,133.8833,15.98,36,0,5.85,AU,1717732089
18,18,kununurra,-15.7667,128.7333,23.63,24,0,7.59,AU,1717732089
21,21,ust'-kut,56.7938,105.7672,17.31,27,32,2.56,RU,1717732090
29,29,ntcheu,-14.8203,34.6359,15.35,64,1,0.39,MW,1717732091
...,...,...,...,...,...,...,...,...,...,...
513,513,sidrolandia,-20.9319,-54.9614,19.18,49,0,2.30,BR,1717732163
534,534,bengkulu,-3.8004,102.2655,29.23,70,10,2.16,ID,1717732166
559,559,pokhara,28.2333,83.9833,27.35,50,27,2.15,NP,1717732170
565,565,rikaze,29.2500,88.8833,16.63,27,17,1.47,CN,1717732171


## Clean the desired vaction weather data frame to include the locatio, and create new columns for finding corresponding hotels in that location. 

In [6]:
#Clean the hotel city df to exclude the 1st column from above, and just include said columnns..
#And..add a column for the hotel name. 
gotel = perfect_hotel_df.loc[:,['City','Country','Lat','Lng','Humidity']]
gotel['Hotel Name']=''
gotel

,City,Country,Lat,Lng,Humidity,Hotel Name
7,oskaloosa,US,41.2964,-92.6444,48,
16,alice springs,AU,-23.7000,133.8833,36,
18,kununurra,AU,-15.7667,128.7333,24,
21,ust'-kut,RU,56.7938,105.7672,27,
29,ntcheu,MW,-14.8203,34.6359,64,
...,...,...,...,...,...,...
513,sidrolandia,BR,-20.9319,-54.9614,49,
534,bengkulu,ID,-3.8004,102.2655,70,
559,pokhara,NP,28.2333,83.9833,50,
565,rikaze,CN,29.2500,88.8833,27,


## Use Geoapify to find hotel names in the geographic locations set in our desired vacation weather data frame, and add the names to the 'gotel' data frame. 

In [7]:
# Set the different parameters to search for a hotel in the cities in the df
radius = 10000
categories='accommodation.hotel'
limit=1

gotel_params = {'categories':categories,
          'apiKey':geoapify_key,'limit':limit}

print("Searching for hotels now...")

# Iterate through the gotel DataFrame to pull out laditude, longitude, and create bias filter to find hotels within our
#...desired radius. 
for index, row in gotel.iterrows():
    latitude=gotel.loc[index,'Lat']
    longitude=gotel.loc[index,'Lng']
    gotel_params["filter"] = f"circle:{longitude},{latitude},{radius}"
    gotel_params["bias"] = f"proximity:{longitude},{latitude}"
    
    # create the base geoapify url
    base_url = "https://api.geoapify.com/v2/places"

#using the parameters in our gotel_params dictionary that are filtered with location preference, make an api request. 
    gotel_addy = requests.get(base_url,gotel_params)
    
    gotel_addy = requests.get(base_url,gotel_params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        gotel.loc[index, "Hotel Name"] = gotel_addy["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        gotel.loc[index, "Hotel Name"] = "Sorry, no hotel found"
        
    # Log the search results
    print(f"{gotel.loc[index, 'City']} - nearest hotel: {gotel.loc[index, 'Hotel Name']}")

# Display sample data
gotel

Searching for hotels now...
oskaloosa - nearest hotel: Fairfield Inn & Suites by Marriott Oskaloosa
alice springs - nearest hotel: Aurora Alice Springs
kununurra - nearest hotel: Sorry, no hotel found
ust'-kut - nearest hotel: Домашний очаг
ntcheu - nearest hotel: Sorry, no hotel found
hassi messaoud - nearest hotel: AMC
avarua - nearest hotel: Paradise Inn
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
hamilton - nearest hotel: North Vista Manor
zuwarah - nearest hotel: فندق الدلفين
tamanrasset - nearest hotel: فندق الأمان
reyes - nearest hotel: Hotel Santa Rosa
west island - nearest hotel: Cocos Village Bungalows
uruacu - nearest hotel: Sorry, no hotel found
al jawf - nearest hotel: Sorry, no hotel found
shingu - nearest hotel: ホテルニューパレス
wenatchee - nearest hotel: Hilton Garden Inn
carnarvon - nearest hotel: Sorry, no hotel found
brookings - nearest hotel: Quality Inn
emerald - nearest hotel: Emerald Central Hotel
sarkand - nearest hotel: Сарканд
lander - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
7,oskaloosa,US,41.2964,-92.6444,48,Fairfield Inn & Suites by Marriott Oskaloosa
16,alice springs,AU,-23.7000,133.8833,36,Aurora Alice Springs
18,kununurra,AU,-15.7667,128.7333,24,"Sorry, no hotel found"
21,ust'-kut,RU,56.7938,105.7672,27,Домашний очаг
29,ntcheu,MW,-14.8203,34.6359,64,"Sorry, no hotel found"
...,...,...,...,...,...,...
513,sidrolandia,BR,-20.9319,-54.9614,49,"Sorry, no hotel found"
534,bengkulu,ID,-3.8004,102.2655,70,Dena Hotel
559,pokhara,NP,28.2333,83.9833,50,Hotel Annapurna View Sarangkot
565,rikaze,CN,29.2500,88.8833,27,Gesar Hotel


# Joe's Optimum Vaction Weather Hotels

## Use hvplot to map the locations of personalized hotels fit to the users(me) weather standards. 
#### Turns out I have semi stringent vactaion weather standards!

In [8]:

%%capture --no-display

# Configure the map plot
heaven_finder=gotel.hvplot.points('Lng','Lat',s='Humidity',color='City',alpha=1,geo=True,tiles=True,xlabel='Longitude',ylabel='Latitude',
                                   xlim=(-180,180),ylim=(-60,60),hover_cols=['Country','Hotel Name'])


# Display the map
heaven_finder

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)